# Introduction

Copyright

Distribution

In order to use the code the following libraries must be installed:
1. Pandas

In [1]:
import pandas as pd
import numpy as np
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import dwave.inspector
import dwavebinarycsp


# Input and Preprocessing

In this section all data input is entered, validated and preprocessed in order to meet further requirements. 

### Input data file

In this section the following input is proceeded:
1. Input data file with information on ferry routes with
    - From: route start port
    - To: route end port
    - Distance: shortest direct distance between departure and arrival port

In [2]:
df_ports = pd.read_csv("sample_data_harbours_utf8.csv", sep = ";")
df_ports

,Column1,Bremerhaven,Brunsbüttel,Emden,Hamburg,Kiel
0,Bremerhaven,0,NaN,137.0,NaN,135.0
1,Brunsbüttel,81,0.0,NaN,36.0,54.0
2,Emden,137,NaN,0.0,NaN,NaN
3,Hamburg,117,36.0,NaN,0.0,90.0
4,Kiel,135,NaN,NaN,NaN,0.0


### Formatting of connection data

In order to formulate the problem in a format accessible by the D-Wave libraries the matrix data is restructured to contain the information in an unstacked format. 

In [3]:
# rename 'Column1' to 'From'
df_ports=df_ports.rename(columns={'Column1':"From"})

# transform end harbours in columns to rows
df_routes= pd.melt(df_ports,id_vars=['From'],var_name="To",value_name='Distance')

# filter NaN-values
df_routes=df_routes[df_routes["Distance"].notnull()]

# filter out 0.0 values
df_routes=df_routes[df_routes.Distance != 0]

#reset index
df_routes=df_routes.reset_index(drop=True)
df_routes_withoutStardEnd=df_routes
df_routes

,From,To,Distance
0,Brunsbüttel,Bremerhaven,81.0
1,Emden,Bremerhaven,137.0
2,Hamburg,Bremerhaven,117.0
3,Kiel,Bremerhaven,135.0
4,Hamburg,Brunsbüttel,36.0
5,Bremerhaven,Emden,137.0
6,Brunsbüttel,Hamburg,36.0
7,Bremerhaven,Kiel,135.0
8,Brunsbüttel,Kiel,54.0
9,Hamburg,Kiel,90.0


### User input and validation of departure and destination harbour 

In this section the manual input is requested and validated by the following criteria:
1. departure and destination harbour are part of the input 

If input is invalid, the according error message is displayed.

In [5]:
#Check whether the departure harbour is element of the input data
while True:
    #Entering departure harbour
    departure=input(str("Please enter departure port:"))
    depature='Brunsbüttel'
    if not departure in df_routes['From'].values:
        print("Entered port is not defined as a depature port in the input file. Please enter valid departure port.")
        continue
    else:
        break

#Check whether the destination harbour is element of the input data
while True:
    #Entering destination harbour
    #destination=input(str("Please enter destination port:"))
    destination='Kiel'
    if not destination in df_routes['To'].values:
        print("Entered port is not defined as a destination port in the input file. Please enter destination valid port.")
        continue
    if destination == departure:
        print("Entered destination port is equal to departure port. Please enter valid destination port.")
        continue
    else:
        #we're happy with the value given.
        #we're ready to exit the loop.
        break


KeyboardInterrupt: Interrupted by user

In [4]:
depature='Brunsbüttel'
destination='Kiel'

### Data enhancement of connection data

A problem can be formulated in different ways in order to provide the quantum computer with it. In this example, the problem is formulated as a binary quadratic model (BQM) in the form of a QUBO. The mathematical phrase can be attained by a support class called Constraint Satisfaction Problem (CSP) or can directly be input. In this case, the problem will first be represented by a CSP. The CSP constitutes of 
1. all variables and 
2. constraints on the variables. 

As the variables in this example will represent the routes between two ports a set of support variables is needed in order to represent the entered departure and destination port. Therefor, artificial routes from "start" port to each real port and to "end" port from each real port are created in the following code. 


In [5]:
df_starts = pd.DataFrame({'From':'start', 'To':df_routes['From'].unique(),'Distance':0})
df_ends = pd.DataFrame({'From':df_routes['To'].unique(), 'To':'end','Distance':0})
df_routes= pd.concat([df_starts,df_routes,df_ends])
df_routes

,From,To,Distance
0,start,Brunsbüttel,0.0
1,start,Emden,0.0
2,start,Hamburg,0.0
3,start,Kiel,0.0
4,start,Bremerhaven,0.0
0,Brunsbüttel,Bremerhaven,81.0
1,Emden,Bremerhaven,137.0
2,Hamburg,Bremerhaven,117.0
3,Kiel,Bremerhaven,135.0
4,Hamburg,Brunsbüttel,36.0


# Data Preparation for Quantum Computing (Definition of Classes)

In this section, all methods are defined that are later used to
1. create Contraint Satisfaction Problem (CSP)
2. formulate Binary Quadratic Model (BQM) as QUBO. 

### Basic functions

In [8]:
def sum_to_two_or_zero(*args):
        """Checks to see if the args sum to either 0 or 2.
        """
        sum_value = sum(args)
        return sum_value in [0, 2]
        
def sum_smaller_equal_one(*args):
        """Checks to see if the args sum to either smaller or equal to 1.
        """
        sum_value = sum(args)
        return sum_value in [0, 1]

def get_labels(dataframe):
        """Returns a list of labels from a Dataframe of the format of the input file"""
        labels=(dataframe['From']+'-'+dataframe['To']+'-'+dataframe['Distance'].astype(str)).values.tolist()
        return labels

### Definition class Ferry

In [9]:
class Ferryarea:
    def __init__(self,start,end,routes):
        """ Initializes object of Type Ferryarea"""
        #Instantiate
        self.start=start
        self.end=end
        self.df_routes = routes#df_routes.values.tolist()
        self.ports = pd.DataFrame({'ports':np.append(df_routes['To'].unique(),df_routes['From'].unique())}).drop_duplicates().values.tolist()
        self.csp = dwavebinarycsp.ConstraintSatisfactionProblem(dwavebinarycsp.BINARY)
        
    def _get_csp(self):
        return self.csp    
    def _apply_valid_routes_constraint(self):
        """Creates constraints defining that every entered port has to be exited as well and that every port can only be entered and exited through one route."""

        for port in self.ports:
            #assign to directions Dataframe a Dataframe containing only direction to and from the port
            df_directions_from= self.df_routes[self.df_routes['From']==port[0]]
            df_directions_to= self.df_routes[self.df_routes['To']==port[0]]
            df_directions= pd.concat([df_directions_to,df_directions_from])
            
            #creates sets of labels as basis for constraint defintion
            directions_all = set(get_labels(df_directions))
            
            #set((df_directions['From']+'-'+df_directions['To']+'-'+df_directions['Distance'].astype(str)).values.tolist())
            directions_to= set(get_labels(df_directions_to))
            #set((df_directions_to['From']+'-'+df_directions_to['To']+'-'+df_directions_to['Distance'].astype(str)).values.tolist())
            directions_from= set(get_labels(df_directions_from))
            #set((df_directions_from['From']+'-'+df_directions_from['To']+'-'+df_directions_from['Distance'].astype(str)).values.tolist())
              
            
            #add constraint sum_to_two_or_zero
            self.csp.add_constraint(sum_to_two_or_zero,directions_all)

            #add constraint 'To' sum_to_zero_or_one
            self.csp.add_constraint(sum_smaller_equal_one,directions_to)

            #add constraint 'From' sum_to_zero_or_one
            self.csp.add_constraint(sum_smaller_equal_one, directions_from)
            
        print(self.csp.constraints)
    def _set_start_and_end(self):
        """Sets the values of the departure and destination port to 1 and all other start and end possibilities to 0
        """ #all other possibilies do not have to be 0. However, this is expected to descrease calculation time

        #select all start and end routes
        df_routes_to_fix= pd.concat([self.df_routes[self.df_routes['From']=='start'],self.df_routes[self.df_routes['To']=='end']])
        print(df_routes_to_fix)
        count=0

        for i in df_routes_to_fix.values:
            
            #print("type:", type(df_routes_to_fix[count]))
            print("label of:", df_routes_to_fix.iloc[count])

            label= get_labels(df_routes_to_fix.iloc[[count]])
            print('Label:',label)
            if (df_routes_to_fix['To'].iloc[count])==self.start:
                #sets departure to 1
                self.csp.fix_variable(label[0],1)
            elif (df_routes_to_fix['From'].iloc[count])==self.end:
                #sets destination to 1
                self.csp.fix_variable(label[0],1)
            else:
                #sets departure and destination to 0
                self.csp.fix_variable(label[0],0)
            count=count+1
        #set start
            #self.csp.fix_variable('start-'+self.start+'-0.0',1)

        #set end
            #self.csp.fix_variable(self.end+'-end-0.0',1)

        #set all other starts and ends to 0
       


### Initialization of Ferryarea

In [10]:
f=Ferryarea(start=departure,end=destination,routes=df_routes)
f._apply_valid_routes_constraint()
f._set_start_and_end()
csp= f._get_csp()

[Constraint.from_configurations(frozenset({(0, 0, 0, 0, 1, 1), (0, 0, 1, 0, 1, 0), (0, 0, 1, 1, 0, 0), (0, 0, 0, 1, 0, 1), (1, 0, 0, 1, 0, 0), (0, 0, 0, 0, 0, 0), (1, 1, 0, 0, 0, 0), (0, 1, 0, 0, 1, 0), (0, 1, 0, 1, 0, 0), (0, 1, 1, 0, 0, 0), (0, 1, 0, 0, 0, 1), (1, 0, 1, 0, 0, 0), (0, 0, 1, 0, 0, 1), (1, 0, 0, 0, 0, 1), (1, 0, 0, 0, 1, 0), (0, 0, 0, 1, 1, 0)}), ('Brunsbüttel-Kiel-54.0', 'Brunsbüttel-Bremerhaven-81.0', 'Brunsbüttel-Hamburg-36.0', 'start-Brunsbüttel-0.0', 'Hamburg-Brunsbüttel-36.0', 'Brunsbüttel-end-0.0'), Vartype.BINARY, name='Constraint'), Constraint.from_configurations(frozenset({(0, 1), (1, 0), (0, 0)}), ('Hamburg-Brunsbüttel-36.0', 'start-Brunsbüttel-0.0'), Vartype.BINARY, name='Constraint'), Constraint.from_configurations(frozenset({(0, 0, 0, 1), (0, 0, 1, 0), (0, 1, 0, 0), (0, 0, 0, 0), (1, 0, 0, 0)}), ('Brunsbüttel-Bremerhaven-81.0', 'Brunsbüttel-Kiel-54.0', 'Brunsbüttel-Hamburg-36.0', 'Brunsbüttel-end-0.0'), Vartype.BINARY, name='Constraint'), Constraint.from_c

UnsatError: fixing Kiel-end-0.0 to 1 makes this constraint unsatisfiable

# Creation of BQM and Transfer to Quantum Sampler

In [ ]:
# Create BQM
bqm = dwavebinarycsp.stitch(csp)

In [ ]:
# Add Penalty
penalty=0.01
for v in bqm.variables:
    split_v = v.split(‘-’)
    bqm.add_variable(v, (penalty*split_v[2]))


In [ ]:
# define Sampler
sampler = EmbeddingComposite(DWaveSampler())


In [ ]:
# run BQM on QPU
result = sampler.sample(bqm,
                        num_reads=100,
                        chain_strength=1,
                        label='ferrylines')

In [ ]:
# visualize result
print(result.first.sample)

# Shortest Way with Dijkstra-Algorithm

Dijkstra Algorithm

In [6]:
# define Graph with distances for every Point
graph={}
for index,port in df_ports.iterrows():
    routes=df_routes_withoutStardEnd[df_routes_withoutStardEnd['From']==port['From']]
    distances={}
    for i,r in routes.iterrows():
        distances[r['To']] = r['Distance']
    graph[port['From']] =distances
print(graph)


{'Bremerhaven': {'Emden': 137.0, 'Kiel': 135.0}, 'Brunsbüttel': {'Bremerhaven': 81.0, 'Hamburg': 36.0, 'Kiel': 54.0}, 'Emden': {'Bremerhaven': 137.0}, 'Hamburg': {'Bremerhaven': 117.0, 'Brunsbüttel': 36.0, 'Kiel': 90.0}, 'Kiel': {'Bremerhaven': 135.0}}


In [7]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [9]:
queue = [depature]
d = {node: {"shortest distance":float("inf"), "previous":None} for node in graph}
d[depature]["shortest distance"] = 0
while queue:
    current = queue.pop(0)
    shortest_distance = d[current]["shortest distance"]
    for neighbour in graph[current]:

        dist_to_neighbour = graph[current][neighbour]

        if shortest_distance + dist_to_neighbour < d[neighbour]["shortest distance"]:

            d[neighbour] = {
                "shortest distance": shortest_distance + dist_to_neighbour,
                "previous": current
            }
            queue.append(neighbour)
print(d)
pp.pprint(d)
print()

{'Bremerhaven': {'shortest distance': 81.0, 'previous': 'Brunsbüttel'}, 'Brunsbüttel': {'shortest distance': 0, 'previous': None}, 'Emden': {'shortest distance': 218.0, 'previous': 'Bremerhaven'}, 'Hamburg': {'shortest distance': 36.0, 'previous': 'Brunsbüttel'}, 'Kiel': {'shortest distance': 54.0, 'previous': 'Brunsbüttel'}}
{   'Bremerhaven': {'previous': 'Brunsbüttel', 'shortest distance': 81.0},
    'Brunsbüttel': {'previous': None, 'shortest distance': 0},
    'Emden': {'previous': 'Bremerhaven', 'shortest distance': 218.0},
    'Hamburg': {'previous': 'Brunsbüttel', 'shortest distance': 36.0},
    'Kiel': {'previous': 'Brunsbüttel', 'shortest distance': 54.0}}



In [10]:
current = destination
path = [current]
distance_sum=0.0
while current != depature:
    distance_sum = distance_sum+d[current]["shortest distance"]
    current = d[current]["previous"]
    path.append(current)

path = path[::-1]
print('Distance: '+str(distance_sum)+' Pfad: '+str(path))

Distance: 54.0 Pfad: ['Brunsbüttel', 'Kiel']
